<a href="https://colab.research.google.com/github/AmalNammouchi/desktop/blob/master/data_proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as  pd 
import numpy as np 
import matplotlib.pyplot as plt 
from  datetime import datetime

In [0]:
# load data 
train=pd.read_csv("train.csv")
sample_sub=pd.read_csv("Sample_sub.csv")

In [778]:
sample_sub.head()

,ID,Capture_Number
0,CaptureSite_0_201901,7
1,CaptureSite_0_201902,1
2,CaptureSite_0_201903,5
3,CaptureSite_0_201904,2
4,CaptureSite_0_201905,3


In [779]:
train.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,Tag_2,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,None,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,None,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,None,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,None,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,None,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [0]:
# split Id to Year woy and CaptureSite
sample_sub["year_woy"]=(sample_sub.ID.apply(lambda x: x.split("_")[-1])).astype(int)
sample_sub["CaptureSite"]=sample_sub.ID.apply(lambda x: ("_").join(x.split("_")[0:-1]))

In [781]:
# Create Time features from Date_TimeCaught 
train["Date_TimeCaught"]=pd.to_datetime(train["Date_TimeCaught"])
print(train.Date_TimeCaught.min(),train.Date_TimeCaught.max())
train["year"]=train.Date_TimeCaught.dt.year
train["week_of_year"]=train.Date_TimeCaught.dt.weekofyear
train["year_woy"]=train.year*100+train.week_of_year

1998-04-17 00:00:00 2018-12-31 00:00:00


In [782]:
keys=pd.concat([train[["year_woy","CaptureSite"]],sample_sub[["year_woy","CaptureSite"]]])
# keys.sort_values(["CaptureSite","year_woy"],inplace=True)
CaptureSite_min_year_woy=keys.groupby("CaptureSite").year_woy.min().rename("year_woy").reset_index()
CaptureSite_min_year_woy.head()

,CaptureSite,year_woy
0,CaptureSite_0,200051
1,CaptureSite_1,200123
2,CaptureSite_10,200050
3,CaptureSite_11,199828
4,CaptureSite_12,200107


In [0]:
range_year_woy=pd.DataFrame()
range_year_woy["Date_TimeCaught"]=pd.date_range(start=train.Date_TimeCaught.min(),end=datetime(2019,10,31))
range_year_woy["year"]=range_year_woy.Date_TimeCaught.dt.year
range_year_woy["week_of_year"]=range_year_woy.Date_TimeCaught.dt.weekofyear
range_year_woy["year_woy"]=range_year_woy.year*100+range_year_woy.week_of_year
range_year_woy.drop_duplicates(["year_woy"],inplace=True)

In [784]:
final_data=[]
for site , year_woy in zip(CaptureSite_min_year_woy.CaptureSite.values,CaptureSite_min_year_woy.year_woy.values) :

    one_site_df=range_year_woy[range_year_woy.year_woy>=year_woy]
    one_site_df["CaptureSite"]=site
    final_data.append(one_site_df)
final_data=pd.concat(final_data)
len(final_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


28048

In [0]:
Target=train.groupby(["year_woy","CaptureSite"]).CaptureSite.count().rename("Capture_Number").reset_index()
final_data=final_data.merge(Target,on=["year_woy","CaptureSite"],how="left")
final_data.Capture_Number.fillna(0,inplace=True)

In [786]:
final_data.Capture_Number.fillna(0).value_counts(True)

0.0     0.716308
1.0     0.147461
2.0     0.059469
3.0     0.030626
4.0     0.015759
5.0     0.009199
6.0     0.005990
7.0     0.005455
8.0     0.002603
9.0     0.002104
10.0    0.001604
11.0    0.000998
12.0    0.000606
14.0    0.000535
13.0    0.000499
15.0    0.000214
16.0    0.000178
17.0    0.000107
28.0    0.000036
22.0    0.000036
19.0    0.000036
35.0    0.000036
20.0    0.000036
21.0    0.000036
18.0    0.000036
23.0    0.000036
Name: Capture_Number, dtype: float64

In [787]:
#final_data=final_data[(final_data.year.isin([]))]
final_data.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0


In [0]:
train=final_data[final_data.year<2019]
test=final_data[final_data.year==2019]

In [789]:
train.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0


In [0]:
data=pd.read_csv('train.csv')

In [791]:
data.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,Tag_2,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,None,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,None,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,None,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,None,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,None,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [0]:
col1 = pd.DataFrame(data.groupby('CaptureSite').agg({'Rescue_ID':['count']}))
col1.columns = [col[0]+"_"+col[1] for col in col1.columns.tolist()]
col1.reset_index(inplace=True)
col1.head()
train=train.merge(col1,how='left',on='CaptureSite')

In [0]:
col1 = pd.DataFrame(data.groupby('CaptureSite').agg({'Researcher':['count']}))
col1.columns = [col[0]+"_"+col[1] for col in col1.columns.tolist()]
col1.reset_index(inplace=True)

In [794]:
col1.head()

,CaptureSite,Researcher_count
0,CaptureSite_0,95
1,CaptureSite_1,793
2,CaptureSite_10,464
3,CaptureSite_11,312
4,CaptureSite_12,573


In [0]:
train=train.merge(col1,how='left',on='CaptureSite')



In [796]:
train.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number,Rescue_ID_count,Researcher_count
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0,95,95
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0,95,95
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0,95,95
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0,95,95
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0,95,95


In [0]:
col2 = pd.DataFrame(data.groupby('CaptureSite').agg({'Fisher':['count']}))
col2.columns = [col[0]+"_"+col[1] for col in col2.columns.tolist()]
col2.reset_index(inplace=True)

In [798]:
col2.head()

,CaptureSite,Fisher_count
0,CaptureSite_0,95
1,CaptureSite_1,793
2,CaptureSite_10,464
3,CaptureSite_11,312
4,CaptureSite_12,573


In [0]:
train=train.merge(col2,how='left',on='CaptureSite')

In [0]:
col3 = pd.DataFrame(data.groupby('CaptureSite').agg({'Species':['count']}))
col3.columns = [col[0]+"_"+col[1] for col in col3.columns.tolist()]
col3.reset_index(inplace=True)

In [801]:
col3.head()

,CaptureSite,Species_count
0,CaptureSite_0,95
1,CaptureSite_1,793
2,CaptureSite_10,464
3,CaptureSite_11,312
4,CaptureSite_12,573


In [0]:
train=train.merge(col3,how='left',on='CaptureSite')

In [803]:
train.head()

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number,Rescue_ID_count,Researcher_count,Fisher_count,Species_count
0,2000-01-01,2000,52,200052,CaptureSite_0,0.0,95,95,95,95
1,2000-12-18,2000,51,200051,CaptureSite_0,1.0,95,95,95,95
2,2001-01-01,2001,1,200101,CaptureSite_0,0.0,95,95,95,95
3,2001-01-08,2001,2,200102,CaptureSite_0,0.0,95,95,95,95
4,2001-01-15,2001,3,200103,CaptureSite_0,0.0,95,95,95,95


In [0]:
from sklearn.model_selection import train_test_split
columns=train.columns

In [805]:
columns

Index(['Date_TimeCaught', 'year', 'week_of_year', 'year_woy', 'CaptureSite',
       'Capture_Number', 'Rescue_ID_count', 'Researcher_count', 'Fisher_count',
       'Species_count'],
      dtype='object')

In [0]:
col1 = pd.DataFrame(data.groupby('CaptureSite').agg({'Researcher':['count']}))
col1.columns = [col[0]+"_"+col[1] for col in col1.columns.tolist()]
col1.reset_index(inplace=True)
test=test.merge(col1,how='left',on='CaptureSite')
col2 = pd.DataFrame(data.groupby('CaptureSite').agg({'Fisher':['count']}))
col2.columns = [col[0]+"_"+col[1] for col in col2.columns.tolist()]
col2.reset_index(inplace=True)
test=test.merge(col2,how='left',on='CaptureSite')
col3 = pd.DataFrame(data.groupby('CaptureSite').agg({'Species':['count']}))
col3.columns = [col[0]+"_"+col[1] for col in col3.columns.tolist()]
col3.reset_index(inplace=True)
test=test.merge(col3,how='left',on='CaptureSite')
col3 = pd.DataFrame(data.groupby('CaptureSite').agg({'Rescue_ID':['count']}))
col3.columns = [col[0]+"_"+col[1] for col in col3.columns.tolist()]
col3.reset_index(inplace=True)
test=test.merge(col3,how='left',on='CaptureSite')


In [0]:
from sklearn import preprocessing
for f in train.columns:
    if train[f].dtype=='object'  : 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values))  
        train[f] = lbl.transform(list(train[f].values))

In [0]:
from sklearn import preprocessing
for f in test.columns:
    if test[f].dtype=='object'  : 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test[f].values))  
        test[f] = lbl.transform(list(test[f].values))

In [809]:
train.columns

Index(['Date_TimeCaught', 'year', 'week_of_year', 'year_woy', 'CaptureSite',
       'Capture_Number', 'Rescue_ID_count', 'Researcher_count', 'Fisher_count',
       'Species_count'],
      dtype='object')

In [0]:

X_train=train[['year', 'week_of_year', 'year_woy', 'Researcher_count', 'Fisher_count', 'Species_count','Rescue_ID_count', 'CaptureSite']]
y_train=train['Capture_Number']

#X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [811]:
train.dtypes

Date_TimeCaught     datetime64[ns]
year                         int64
week_of_year                 int64
year_woy                     int64
CaptureSite                  int64
Capture_Number             float64
Rescue_ID_count              int64
Researcher_count             int64
Fisher_count                 int64
Species_count                int64
dtype: object

In [0]:
#from sklearn.linear_model import LinearRegression
#model1 = LinearRegression()
#model1.fit(X_train,y_train)


In [812]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:28:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:

#X_test=test[[ 'year', 'week_of_year', 'year_woy', 'CaptureSite'
       , 'Researcher_count', 'Fisher_count', 'Species_count','Rescue_ID_count']]
#y_test=test['Capture_Number']

X_test=test[['year', 'week_of_year', 'year_woy', 'Researcher_count', 'Fisher_count', 'Species_count','Rescue_ID_count', 'CaptureSite']]
y_test=test['Capture_Number']
#X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [0]:
y_pred = my_model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [815]:

#from sklearn.metrics import mean_absolute_error
#print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,predictions)

6.487460815047022

In [816]:
import lightgbm as lgb
param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, X_train, num_round, valid_sets=[y_train], early_stopping_rounds=10, verbose_eval=False)


TypeError: ignored

In [0]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv" , index=False)
    return FileLink(submission_name+".csv")

In [0]:
#Sample_sub = pd.read_csv('Sample_sub')
Sample_sub = pd.read_csv('Sample_sub.csv')

In [0]:
Sample_sub['Capture_Number'] =y_pred


In [0]:
create_submission(Sample_sub ,  'sub3')

In [0]:
data.isna().sum()

In [0]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['CaptureSite']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = train[cat_features].apply(encoder.fit_transform)
encoded.head(-10)
# Since ks and encoded have the same index and I can easily join them
train =train[['Capture_Number' , 'year', 'week_of_year', 'year_woy',
              'Researcher_count', 'Fisher_count', 'Species_count','Rescue_ID_count']].join(encoded)


# Apply the label encoder to each column
encoded = test[cat_features].apply(encoder.fit_transform)
encoded.head(-10)
# Since ks and encoded have the same index and I can easily join them
test =test[['Capture_Number' , 'year', 'week_of_year', 'year_woy',
              'Researcher_count', 'Fisher_count', 'Species_count','Rescue_ID_count']].join(encoded)
train.head()